In [4]:
ims = []/
ims += [['str']]
ims += [['str3']]
print(ims)

[['str'], ['str3']]


In [13]:
import numpy as np

In [14]:
a = [[11,22,33,44,0.9], [21,20,33,44,0.9], [11,23,33,44,0.9]]
a = np.array(a)
a.shape

(3, 5)

In [20]:
c= a[a[:,1].argsort()]
c

array([[21. , 20. , 33. , 44. ,  0.9],
       [11. , 22. , 33. , 44. ,  0.9],
       [11. , 23. , 33. , 44. ,  0.9]])

In [21]:
b = c[::-1]

In [22]:
b

array([[11. , 23. , 33. , 44. ,  0.9],
       [11. , 22. , 33. , 44. ,  0.9],
       [21. , 20. , 33. , 44. ,  0.9]])

In [12]:
for r in a:
    print(r)

[11.  22.  33.  44.   0.9]
[21.  22.  33.  44.   0.9]
[11.  22.  33.  44.   0.9]


In [ ]:
def fprint(fp, wstr):
    if fp is None:
        pass
    else:
        fp.write(wstr)

In [29]:
box_info = {0:[0], 1:[0],2:[0],3:[0],4:[0]}

In [43]:
def insert_box_info__(boxs, cid, bid, p_info):
    new_score = p_info[cid]
    if boxs[cid][0] == 0: #no stroed score
        boxs[cid][0] = new_score
        boxs[cid].append([bid, p_info])
        print("Insert new info wo conflict")
        print(boxs)
        return
    else:
        if new_score>boxs[cid][0]: #replace boxes info
            print("conflict 1", boxs)
            revised_info = list(boxs[cid][1][1])
            revised_info[cid] = -1
            unsorted_info = tuple(revised_info)
            unsorted_bid = boxs[cid][1][0]
            unsorted_cid = np.argmax(unsorted_info)
            print("unsorted ", unsorted_cid, unsorted_bid, unsorted_info)
            boxs[cid][0] = new_score
            boxs[cid][1][0] = bid
            boxs[cid][1][1] = p_info
            print(boxs)
            return insert_box_info(boxs,unsorted_cid,unsorted_bid,unsorted_info)
        elif new_score <0:
            print("box slots are full, discard this one bid%d"%bid)
            return
        else:    
            print("conflict 2", boxs)
            revised_info = list(p_info)
            revised_info[cid] = -1    
            revised_info = tuple(revised_info)
            revised_cid = np.argmax(revised_info)
            print("revised", revised_cid, bid, revised_info)
            return insert_box_info(boxs, revised_cid,bid, revised_info)      

In [80]:
def fprint(fw, ostr):
    print(ostr)

In [81]:
def insert_box_info(boxs, boxs_xy, cid, bid, p_info, fw=None):
    new_score = p_info[cid]
    if boxs[cid][0] == 0: #no stroed score
        boxs[cid][0] = new_score
        boxs[cid].append([bid, p_info])
        if boxs_xy[bid]['cid']!= cid:
            print('ERROR! %d-%d-%d'%(bid, cid, boxs_xy[bid]['cid']))
        fprint(fw,"[A] Insert new info wo conflict")
        #fprint(fw,boxs)
        return
    else:
        if new_score>boxs[cid][0]: #replace boxes info
            fprint(fw, "[B] conflict 1")
            revised_info = list(boxs[cid][1][1])
            revised_info[cid] = -1
            revised_info = tuple(revised_info)
            restore_bid = boxs[cid][1][0]
            revised_cid = np.argmax(revised_info)
            boxs_xy[restore_bid]['cid'] = revised_cid
            boxs_xy[restore_bid]['revised'] = True
            fprint(fw, "unsorted %s-%s-%s"%(revised_cid, restore_bid, revised_info))
            boxs[cid][0] = new_score
            boxs[cid][1][0] = bid
            boxs[cid][1][1] = p_info
            #print(boxs)
            return insert_box_info(boxs,boxs_xy,revised_cid, restore_bid,revised_info, fw)
        elif new_score <0:
            boxs_xy[bid]['revised'] = -1 #dont draw this box
            fprint(fw, "[D] box slots are full, discard this one bid%d"%bid)
            return
        else:    
            fprint(fw, "[C]conflict 2")
            revised_info = list(p_info)
            revised_info[cid] = -1    
            revised_info = tuple(revised_info)
            revised_cid = np.argmax(revised_info)
            boxs_xy[bid]['cid'] = revised_cid
            boxs_xy[bid]['revised'] = True

            fprint(fw, "revised %s-%s-%s"%(revised_cid, bid, revised_info))
            return insert_box_info(boxs, boxs_xy,revised_cid,bid, revised_info, fw)      


In [101]:
box_info2 = {0:[0], 1:[0],2:[0],3:[0],4:[0]}
box_xy = {0:{'cid':0, 'revised':0}, 11:{'cid':0, 'revised':0},12:{'cid':0, 'revised':0},13:{'cid':0, 'revised':0},14:{'cid':0, 'revised':0}}
insert_box_info(box_info2,box_xy, 0, 0,(9,8,7,6))

[A] Insert new info wo conflict


In [102]:
insert_box_info(box_info2, box_xy,1, 11,(4,7,5,6))

ERROR! 11-1-0
[A] Insert new info wo conflict


In [103]:
insert_box_info(box_info2, box_xy, 2, 12,(0,1,9,6))

ERROR! 12-2-0
[A] Insert new info wo conflict


In [104]:
insert_box_info(box_info2, box_xy, 1, 13,(3,9,5,2))

[B] conflict 1
unsorted 3-11-(4, -1, 5, 6)
[A] Insert new info wo conflict


In [105]:
insert_box_info(box_info2, box_xy, 1, 14,(3,9,5,2))

[C]conflict 2
revised 2-14-(3, -1, 5, 2)
[C]conflict 2
revised 0-14-(3, -1, -1, 2)
[C]conflict 2
revised 3-14-(-1, -1, -1, 2)
[C]conflict 2
revised 0-14-(-1, -1, -1, -1)
[D] box slots are full, discard this one bid14


In [106]:
box_info2

{0: [9, [0, (9, 8, 7, 6)]],
 1: [9, [13, (3, 9, 5, 2)]],
 2: [9, [12, (0, 1, 9, 6)]],
 3: [6, [11, (4, -1, 5, 6)]],
 4: [0]}

In [107]:
box_xy

{0: {'cid': 0, 'revised': 0},
 11: {'cid': 3, 'revised': True},
 12: {'cid': 0, 'revised': 0},
 13: {'cid': 0, 'revised': 0},
 14: {'cid': 0, 'revised': -1}}

In [31]:
insert_box_info(box_info, 0, 1,(9,8,7,6))

Insert new info wo conflict
{0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [0], 3: [0], 4: [0]}


In [32]:
insert_box_info(box_info, 0, 11,(9,1,7,6))

conflict 2 {0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [0], 3: [0], 4: [0]}
revised 2 11 (-1, 1, 7, 6)
Insert new info wo conflict
{0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [7, [11, (-1, 1, 7, 6)]], 3: [0], 4: [0]}


In [33]:
insert_box_info(box_info, 2, 12,(1,3,8,2))

conflict 1 {0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [7, [11, (-1, 1, 7, 6)]], 3: [0], 4: [0]}
unsorted  3 11 (-1, 1, -1, 6)
{0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [8, [12, (1, 3, 8, 2)]], 3: [0], 4: [0]}
Insert new info wo conflict
{0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [8, [12, (1, 3, 8, 2)]], 3: [6, [11, (-1, 1, -1, 6)]], 4: [0]}


In [35]:
insert_box_info(box_info, 3, 13,(3,1,2,9))

conflict 1 {0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [8, [12, (1, 3, 8, 2)]], 3: [6, [11, (-1, 1, -1, 6)]], 4: [0]}
unsorted  1 11 (-1, 1, -1, -1)
{0: [9, [1, (9, 8, 7, 6)]], 1: [0], 2: [8, [12, (1, 3, 8, 2)]], 3: [9, [13, (3, 1, 2, 9)]], 4: [0]}
Insert new info wo conflict
{0: [9, [1, (9, 8, 7, 6)]], 1: [1, [11, (-1, 1, -1, -1)]], 2: [8, [12, (1, 3, 8, 2)]], 3: [9, [13, (3, 1, 2, 9)]], 4: [0]}


In [44]:
insert_box_info(box_info, 0, 15,(7,0,7,7))

conflict 2 {0: [9, [1, (9, 8, 7, 6)]], 1: [1, [11, (-1, 1, -1, -1)]], 2: [8, [12, (1, 3, 8, 2)]], 3: [9, [13, (3, 1, 2, 9)]], 4: [0]}
revised 2 15 (-1, 0, 7, 7)
conflict 2 {0: [9, [1, (9, 8, 7, 6)]], 1: [1, [11, (-1, 1, -1, -1)]], 2: [8, [12, (1, 3, 8, 2)]], 3: [9, [13, (3, 1, 2, 9)]], 4: [0]}
revised 3 15 (-1, 0, -1, 7)
conflict 2 {0: [9, [1, (9, 8, 7, 6)]], 1: [1, [11, (-1, 1, -1, -1)]], 2: [8, [12, (1, 3, 8, 2)]], 3: [9, [13, (3, 1, 2, 9)]], 4: [0]}
revised 1 15 (-1, 0, -1, -1)
conflict 2 {0: [9, [1, (9, 8, 7, 6)]], 1: [1, [11, (-1, 1, -1, -1)]], 2: [8, [12, (1, 3, 8, 2)]], 3: [9, [13, (3, 1, 2, 9)]], 4: [0]}
revised 0 15 (-1, -1, -1, -1)
box slots are full, discard this one bid15


In [36]:
box_info

{0: [9, [1, (9, 8, 7, 6)]],
 1: [1, [11, (-1, 1, -1, -1)]],
 2: [8, [12, (1, 3, 8, 2)]],
 3: [9, [13, (3, 1, 2, 9)]],
 4: [0]}

In [42]:
box_info

{0: [9, [1, (9, 8, 7, 6)]],
 1: [1, [11, (-1, 1, -1, -1)]],
 2: [8, [12, (1, 3, 8, 2)]],
 3: [9, [13, (3, 1, 2, 9)]],
 4: [0]}